# Sniffing packets
- sniff() function allows us to capture filtered packets of interest
- equivalent to tcpdump and wireshark
- also has built-in wireshark function to parse big and complex packet captures

In [1]:
from scapy.all import *
conf.verb=1
conf.color_theme = RastaTheme()

In [2]:
help(sniff)

Help on function sniff in module scapy.sendrecv:

sniff(*args, **kwargs)
    Sniff packets and return a list of packets.
    
    Args:
        count: number of packets to capture. 0 means infinity.
        store: whether to store sniffed packets or discard them
        prn: function to apply to each packet. If something is returned, it
             is displayed.
             --Ex: prn = lambda x: x.summary()
        session: a session = a flow decoder used to handle stream of packets.
                 e.g: IPSession (to defragment on-the-flow) or NetflowSession
        filter: BPF filter to apply.
        lfilter: Python function applied to each packet to determine if
                 further action may be done.
                 --Ex: lfilter = lambda x: x.haslayer(Padding)
        offline: PCAP file (or list of PCAP files) to read packets from,
                 instead of sniffing them
        timeout: stop sniffing after a given time (default: None).
        L2socket: use the provid

In [3]:
# count argument value will sniff that many packets
pkts = sniff(count=2)

In [4]:
pkts

<Sniffed: TCP:2 UDP:0 ICMP:0 Other:0>

In [5]:
pkts.summary()

Ether / IP / TCP 104.81.94.116:https > 192.168.100.4:42646 PA / Raw
Ether / IP / TCP 192.168.100.4:42646 > 104.81.94.116:https A


## Sniffing ping/icmp packets 
### Generate icmp packets using ping
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [6]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=2, filter="icmp")

In [7]:
pkts

<Sniffed: TCP:0 UDP:0 ICMP:2 Other:0>

In [8]:
pkts.summary()

Ether / IP / ICMP 192.168.100.4 > 216.58.217.46 echo-request 0 / Raw
Ether / IP / ICMP 216.58.217.46 > 192.168.100.4 echo-reply 0 / Raw


In [9]:
pkts.show()

0000 Ether / IP / ICMP 192.168.100.4 > 216.58.217.46 echo-request 0 / Raw
0001 Ether / IP / ICMP 216.58.217.46 > 192.168.100.4 echo-reply 0 / Raw


In [10]:
pkts[0].show()

###[ Ethernet ]### 
  dst= 52:54:00:12:35:00
  src= 08:00:27:f7:e8:f7
  type= IPv4
###[ IP ]### 
     version= 4
     ihl= 5
     tos= 0x0
     len= 84
     id= 9732
     flags= DF
     frag= 0
     ttl= 64
     proto= icmp
     chksum= 0x3e8f
     src= 192.168.100.4
     dst= 216.58.217.46
     \options\
###[ ICMP ]### 
        type= echo-request
        code= 0
        chksum= 0x6dc0
        id= 0x6b8d
        seq= 0x1
###[ Raw ]### 
           load= '\x1f\xc4\x8f]|\x8c\x08\x00\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'



In [11]:
hexdump(pkts[0])

0000  52 54 00 12 35 00 08 00 27 F7 E8 F7 08 00 45 00  RT..5...'.....E.
0010  00 54 26 04 40 00 40 01 3E 8F C0 A8 64 04 D8 3A  .T&.@.@.>...d..:
0020  D9 2E 08 00 6D C0 6B 8D 00 01 1F C4 8F 5D 7C 8C  ....m.k......]|.
0030  08 00 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 14 15  ................
0040  16 17 18 19 1A 1B 1C 1D 1E 1F 20 21 22 23 24 25  .......... !"#$%
0050  26 27 28 29 2A 2B 2C 2D 2E 2F 30 31 32 33 34 35  &'()*+,-./012345
0060  36 37                                            67


## Sniffing arp packets 
### Generate arp packets using arping command
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [12]:
# ping a non-existant private ip or gateway or use arping
pkts = sniff(count=5, filter="arp")

In [13]:
pkts.summary()

Ether / ARP who has 192.168.100.1 says 192.168.100.4 / Padding
Ether / ARP is at 52:54:00:12:35:00 says 192.168.100.1 / Padding
Ether / ARP who has 192.168.100.1 says 192.168.100.4 / Padding
Ether / ARP is at 52:54:00:12:35:00 says 192.168.100.1 / Padding
Ether / ARP who has 192.168.100.1 says 192.168.100.4 / Padding


In [14]:
pkts[0].show()

###[ Ethernet ]### 
  dst= ff:ff:ff:ff:ff:ff
  src= 08:00:27:f7:e8:f7
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= 6
     plen= 4
     op= who-has
     hwsrc= 08:00:27:f7:e8:f7
     psrc= 192.168.100.4
     hwdst= 00:00:00:00:00:00
     pdst= 192.168.100.1
###[ Padding ]### 
        load= '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [15]:
pkts[1].show()

###[ Ethernet ]### 
  dst= 08:00:27:f7:e8:f7
  src= 52:54:00:12:35:00
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= 6
     plen= 4
     op= is-at
     hwsrc= 52:54:00:12:35:00
     psrc= 192.168.100.1
     hwdst= 08:00:27:f7:e8:f7
     pdst= 192.168.100.4
###[ Padding ]### 
        load= '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [16]:
pkts[0].command() # show command to generate the first packet

"Ether(dst='ff:ff:ff:ff:ff:ff', src='08:00:27:f7:e8:f7', type=2054)/ARP(hwtype=1, ptype=2048, hwlen=6, plen=4, op=1, hwsrc='08:00:27:f7:e8:f7', psrc='192.168.100.4', hwdst='00:00:00:00:00:00', pdst='192.168.100.1')/Padding(load=b'\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00')"

## Custom formatted ARP Monitor
- https://thepacketgeek.com/scapy-sniffing-with-custom-actions-part-1/
- run the following code and ping a valid local ip from a terminal to generate 

In [17]:
# check ARP table
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.100.3            ether   08:00:27:0f:41:2e   C                     eth0
192.168.100.1            ether   52:54:00:12:35:00   C                     eth0


In [18]:
# clear arp cache and validate
! ip -s -s neigh flush all

192.168.100.3 dev eth0 lladdr 08:00:27:0f:41:2e used 477/477/442 probes 4 STALE
192.168.100.1 dev eth0 lladdr 52:54:00:12:35:00 used 150/106/75 probes 1 STALE
fe80::5054:ff:fe12:3500 dev eth0 lladdr 52:54:00:12:35:00 router used 1009/1069/1009 probes 0 STALE

*** Round 1, deleting 3 entries ***
*** Flush is complete after 1 round ***


In [20]:
# validate
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.100.1            ether   52:54:00:12:35:00   C                     eth0


In [6]:
from collections import Counter
from scapy.all import ARP, sniff

def arp_display(pkt):
    if pkt[ARP].op == 1: #who-has (request)
        return f"Request: {pkt[ARP].psrc} is asking about {pkt[ARP].pdst}"
    if pkt[ARP].op == 2: #is-at (response)
        return f"*Response: {pkt[ARP].hwsrc} has address {pkt[ARP].psrc}"

sniff(prn=arp_display, filter="arp", store=0, count=4)

Request: 192.168.231.152 is asking about 192.168.231.2
*Response: 00:50:56:f9:21:6d has address 192.168.231.2
Request: 192.168.231.152 is asking about 192.168.231.2
*Response: 00:50:56:f9:21:6d has address 192.168.231.2
Request: 192.168.231.152 is asking about 192.168.231.2
*Response: 00:50:56:f9:21:6d has address 192.168.231.2


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [ ]:
help(sniff)

## Keeping track of number of packets sniffed

In [1]:
from collections import Counter
from scapy.all import sniff

## Create a Packet Counter
packet_counts = Counter()

## Define our Custom Action function
def custom_action(packet):
    # Create tuple of Src/Dst in sorted order
    key = tuple(sorted([packet[0][1].src, packet[0][1].dst]))
    packet_counts.update([key])
    return f"Packet #{sum(packet_counts.values())}: {packet[0][1].src} ==> {packet[0][1].dst}"

## Setup sniff, filtering for IP traffic
sniff(filter="ip", prn=custom_action, count=10)

## Print out packet count per A <--> Z address pair
print("\n".join(f"{f'{key[0]} <--> {key[1]}'}: {count}" for key, count in packet_counts.items()))

Packet #1: 192.168.231.152 ==> 185.199.110.153
Packet #2: 185.199.110.153 ==> 192.168.231.152
Packet #3: 192.168.231.152 ==> 72.5.72.15
Packet #4: 72.5.72.15 ==> 192.168.231.152
Packet #5: 192.168.231.152 ==> 192.229.163.25
Packet #6: 192.229.163.25 ==> 192.168.231.152
Packet #7: 192.168.231.152 ==> 72.21.91.29
Packet #8: 72.21.91.29 ==> 192.168.231.152
Packet #9: 192.168.231.152 ==> 172.64.103.24
Packet #10: 172.64.103.24 ==> 192.168.231.152
185.199.110.153 <--> 192.168.231.152: 2
192.168.231.152 <--> 72.5.72.15: 2
192.168.231.152 <--> 192.229.163.25: 2
192.168.231.152 <--> 72.21.91.29: 2
172.64.103.24 <--> 192.168.231.152: 2


## Importing and Exporting Packets
## PCAP format
- save capture packets to pcap file for use at later time or with different applications

In [7]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=5, filter="icmp")

In [11]:
from scapy.all import wrpcap
wrpcap('icmp.pcap', pkts)

In [20]:
# use scapy's hexdump to see pkts
from scapy.all import hexdump
hexdump(pkts[0])

0000  00 50 56 F9 21 6D 00 0C 29 D7 4D 09 08 00 45 00  .PV.!m..).M...E.
0010  00 54 26 55 40 00 40 01 C4 67 C0 A8 E7 98 C0 A8  .T&U@.@..g......
0020  E7 02 08 00 5A 58 0C 47 00 01 2B 4A 81 5D F0 B4  ....ZX.G..+J.]..
0030  09 00 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 14 15  ................
0040  16 17 18 19 1A 1B 1C 1D 1E 1F 20 21 22 23 24 25  .......... !"#$%
0050  26 27 28 29 2A 2B 2C 2D 2E 2F 30 31 32 33 34 35  &'()*+,-./012345
0060  36 37                                            67


In [21]:
# use bash hexdump
! hexdump -C icmp.pcap

00000000  d4 c3 b2 a1 02 00 04 00  00 00 00 00 00 00 00 00  |................|
00000010  ff ff 00 00 01 00 00 00  2b 4a 81 5d b0 b6 09 00  |........+J.]....|
00000020  62 00 00 00 62 00 00 00  00 50 56 f9 21 6d 00 0c  |b...b....PV.!m..|
00000030  29 d7 4d 09 08 00 45 00  00 54 26 55 40 00 40 01  |).M...E..T&U@.@.|
00000040  c4 67 c0 a8 e7 98 c0 a8  e7 02 08 00 5a 58 0c 47  |.g..........ZX.G|
00000050  00 01 2b 4a 81 5d f0 b4  09 00 08 09 0a 0b 0c 0d  |..+J.]..........|
00000060  0e 0f 10 11 12 13 14 15  16 17 18 19 1a 1b 1c 1d  |................|
00000070  1e 1f 20 21 22 23 24 25  26 27 28 29 2a 2b 2c 2d  |.. !"#$%&'()*+,-|
00000080  2e 2f 30 31 32 33 34 35  36 37 2b 4a 81 5d c7 b5  |./01234567+J.]..|
00000090  09 00 62 00 00 00 62 00  00 00 00 0c 29 d7 4d 09  |..b...b.....).M.|
000000a0  00 50 56 f9 21 6d 08 00  45 00 00 54 97 24 00 00  |.PV.!m..E..T.$..|
000000b0  80 01 53 98 c0 a8 e7 02  c0 a8 e7 98 00 00 62 58  |..S...........bX|
000000c0  0c 47 00 01 2b 4a 81 5d  f0 b4 09 00 08 09

In [17]:
# restore previously saved pcap file
from scapy.all import rdpcap
pkts1 = rdpcap('icmp.pcap')

In [18]:
pkts1.summary()

Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw
Ether / IP / ICMP 192.168.231.2 > 192.168.231.152 echo-reply 0 / Raw
Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw
Ether / IP / ICMP 192.168.231.2 > 192.168.231.152 echo-reply 0 / Raw
Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw


In [26]:
pkts1.show()

0000 Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw
0001 Ether / IP / ICMP 192.168.231.2 > 192.168.231.152 echo-reply 0 / Raw
0002 Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw
0003 Ether / IP / ICMP 192.168.231.2 > 192.168.231.152 echo-reply 0 / Raw
0004 Ether / IP / ICMP 192.168.231.152 > 192.168.231.2 echo-request 0 / Raw


In [24]:
new_pkt = pkts[0]

In [25]:
new_pkt

<Ether  dst=00:50:56:f9:21:6d src=00:0c:29:d7:4d:09 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=84 id=9813 flags=DF frag=0 ttl=64 proto=icmp chksum=0xc467 src=192.168.231.152 dst=192.168.231.2 |<ICMP  type=echo-request code=0 chksum=0x5a58 id=0xc47 seq=0x1 |<Raw  load='+J\x81]\xf0\xb4\t\x00\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567' |>>>>

## open icmp.pcap file in wireshark to analyze